In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.preprocessing import LabelEncoder


In [4]:
TRAIN_DIR = "images/train"
TEST_DIR = "images/test"


In [5]:
def create_dataframe(dir_path):
    image_paths = []
    labels = []

    for label in os.listdir(dir_path):
        label_path = os.path.join(dir_path, label)
        for img_name in os.listdir(label_path):
            image_paths.append(os.path.join(label_path, img_name))
            labels.append(label)
        print(label, "completed")

    return pd.DataFrame({"image": image_paths, "label": labels})


train_df = create_dataframe(TRAIN_DIR)
test_df = create_dataframe(TEST_DIR)


angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
le = LabelEncoder()
le.fit(train_df["label"])

train_df["label"] = le.transform(train_df["label"])
test_df["label"] = le.transform(test_df["label"])

NUM_CLASSES = 7


In [7]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.ToTensor()  # automatically scales to [0,1]
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image"]
        label = self.df.iloc[idx]["label"]

        image = Image.open(img_path)
        image = self.transform(image)

        return image, label


In [8]:
train_dataset = EmotionDataset(train_df)
test_dataset = EmotionDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [9]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),

            nn.Conv2d(128, 256, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),

            nn.Conv2d(256, 512, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),

            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 1 * 1, 512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, NUM_CLASSES)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNN().to(device)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
EPOCHS = 100

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {train_loss:.4f} | Val Acc: {acc:.2f}%")


Epoch [1/100] | Loss: 410.4664 | Val Acc: 24.68%
Epoch [2/100] | Loss: 397.8511 | Val Acc: 33.43%
Epoch [3/100] | Loss: 376.1193 | Val Acc: 38.32%
Epoch [4/100] | Loss: 354.1758 | Val Acc: 44.64%
Epoch [5/100] | Loss: 338.2445 | Val Acc: 46.74%
Epoch [6/100] | Loss: 326.6645 | Val Acc: 48.46%
Epoch [7/100] | Loss: 318.2745 | Val Acc: 49.79%
Epoch [8/100] | Loss: 311.4315 | Val Acc: 52.01%
Epoch [9/100] | Loss: 304.2385 | Val Acc: 53.55%
Epoch [10/100] | Loss: 298.1802 | Val Acc: 54.02%
Epoch [11/100] | Loss: 293.2588 | Val Acc: 54.73%
Epoch [12/100] | Loss: 287.9858 | Val Acc: 55.60%
Epoch [13/100] | Loss: 284.7067 | Val Acc: 56.88%
Epoch [14/100] | Loss: 280.7165 | Val Acc: 56.01%
Epoch [15/100] | Loss: 277.7347 | Val Acc: 56.30%
Epoch [16/100] | Loss: 273.0623 | Val Acc: 56.84%
Epoch [17/100] | Loss: 271.7164 | Val Acc: 57.74%
Epoch [18/100] | Loss: 269.8632 | Val Acc: 57.80%
Epoch [19/100] | Loss: 267.3577 | Val Acc: 58.25%
Epoch [20/100] | Loss: 265.8182 | Val Acc: 58.59%
Epoch [21

In [13]:
torch.save(model.state_dict(), "emotiondetector.pth")


In [14]:
model = EmotionCNN().to(device)
model.load_state_dict(torch.load("emotiondetector.pth", map_location=device))
model.eval()


C:\Users\subha\AppData\Local\Temp\ipykernel_11156\2979743912.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("emotiondetector.pth", map_

EmotionCNN(
  (features): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.4, inplace=False)
    (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.4, inplace=False)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.4, inplace=False)
    (12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Dropout(p=0.4, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
    (3

In [18]:
labels = ['angry','disgust','fear','happy','neutral','sad','surprise']

def predict_image(image_path):
    transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize((48, 48)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, dim=1).item()

    return labels[pred]


image = "D:\\vs\\Subham\\python\\model\\mood classifier\\Face_Emotion_Recognition_Machine_Learning\\images\\test\\angry\\245.jpg"
print("Model prediction:", predict_image(image))


Model prediction: angry
